### Sklearn Examples

In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

RANDOM_SEED = 42
BINARY_LABEL = "is_hate"
CATEGORIES = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

np.random.seed(RANDOM_SEED)  # set random seed for reproducibility

def binarize_labels(df):
    return (df[CATEGORIES].sum(axis=1) > 0).astype(int)

data_train = pd.read_csv("../data/hatespeech/train.csv", index_col=0)
data_train[BINARY_LABEL] = binarize_labels(data_train)

data_test = pd.read_csv("../data/hatespeech/test.csv", index_col=0).join(
    pd.read_csv("../data/hatespeech/test_labels.csv", index_col=0)
)
data_test.drop(data_test[data_test["toxic"] == -1].index, inplace=True)
data_test[BINARY_LABEL] = binarize_labels(data_test)

In [2]:
X_train = data_train["comment_text"]
y_train = data_train["is_hate"]

X_test = data_test["comment_text"]
y_test = data_test["is_hate"]

In [3]:
linear_pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression())
svm_pipeline = make_pipeline(TfidfVectorizer(), SGDClassifier())

In [4]:
linear_pipeline.fit(X_train, y_train)
y_pred = linear_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96     57735
           1       0.66      0.71      0.68      6243

    accuracy                           0.94     63978
   macro avg       0.81      0.84      0.82     63978
weighted avg       0.94      0.94      0.94     63978



In [5]:
svm_pipeline.fit(X_train, y_train)
y_pred = svm_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     57735
           1       0.79      0.52      0.63      6243

    accuracy                           0.94     63978
   macro avg       0.87      0.75      0.80     63978
weighted avg       0.94      0.94      0.93     63978



### SetFit Examples

Attention: Typically requires a GPU to train -> uses small sample of dataset for demonstration

In [6]:
from setfit import SetFitModel, Trainer, TrainingArguments

In [7]:
dataset_train = Dataset.from_pandas(
    # take 10 samples from each class
    data_train.groupby("is_hate").sample(10, random_state=RANDOM_SEED)
)
dataset_test = Dataset.from_pandas(
    # take 100 samples from each class
    data_test.groupby("is_hate").sample(100, random_state=RANDOM_SEED)
)

In [8]:
model = SetFitModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    labels=[0, 1],
).to("cpu")

args = TrainingArguments(
    batch_size=1,
    num_epochs=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    num_iterations=20,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    metric="accuracy",
    column_mapping={"comment_text": "text", "is_hate": "label"}
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [9]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num unique pairs = 800
  Batch size = 1
  Num epochs = 1
  Total optimization steps = 800


  0%|          | 0/800 [00:00<?, ?it/s]

  0%|          | 0/800 [00:00<?, ?it/s]

{'embedding_loss': 0.5138, 'learning_rate': 2.5000000000000004e-07, 'epoch': 0.0}
{'embedding_loss': 0.0554, 'learning_rate': 1.25e-05, 'epoch': 0.06}
{'embedding_loss': 0.0001, 'learning_rate': 1.9444444444444445e-05, 'epoch': 0.12}
{'embedding_loss': 0.0312, 'learning_rate': 1.8055555555555558e-05, 'epoch': 0.19}
{'embedding_loss': 0.0025, 'learning_rate': 1.6666666666666667e-05, 'epoch': 0.25}
{'embedding_loss': 0.0003, 'learning_rate': 1.5277777777777777e-05, 'epoch': 0.31}
{'embedding_loss': 0.0, 'learning_rate': 1.388888888888889e-05, 'epoch': 0.38}
{'embedding_loss': 0.0, 'learning_rate': 1.25e-05, 'epoch': 0.44}
{'embedding_loss': 0.0008, 'learning_rate': 1.1111111111111113e-05, 'epoch': 0.5}
{'embedding_loss': 0.0002, 'learning_rate': 9.722222222222223e-06, 'epoch': 0.56}
{'embedding_loss': 0.0, 'learning_rate': 8.333333333333334e-06, 'epoch': 0.62}
{'embedding_loss': 0.0002, 'learning_rate': 6.944444444444445e-06, 'epoch': 0.69}
{'embedding_loss': 0.0001, 'learning_rate': 5.5

  0%|          | 0/8000 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 800.


{'eval_embedding_loss': 0.2344, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 308.3876, 'train_samples_per_second': 2.594, 'train_steps_per_second': 2.594, 'epoch': 1.0}


***** Running evaluation *****


{'accuracy': 0.775}

In [10]:
y_true = dataset_test["is_hate"]
y_pred_setfit = model.predict(dataset_test["comment_text"])
print(classification_report(y_true, y_pred_setfit))

              precision    recall  f1-score   support

           0       0.87      0.65      0.74       100
           1       0.72      0.90      0.80       100

    accuracy                           0.78       200
   macro avg       0.79      0.78      0.77       200
weighted avg       0.79      0.78      0.77       200



# XGBoost Examplwe

In [12]:
from xgboost import XGBClassifier

xgboost_pipeline = make_pipeline(TfidfVectorizer(), XGBClassifier())
xgboost_pipeline.fit(X_train, y_train)
y_pred = xgboost_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96     57735
           1       0.63      0.70      0.66      6243

    accuracy                           0.93     63978
   macro avg       0.80      0.83      0.81     63978
weighted avg       0.93      0.93      0.93     63978

